In [1]:
import pandas as pd

df=pd.read_csv('dialogs.txt',sep='\t',names=['User','Chatbot'])
print(df)


                                                   User  \
0                                hi, how are you doing?   
1                         i'm fine. how about yourself?   
2                   i'm pretty good. thanks for asking.   
3                     no problem. so how have you been?   
4                      i've been great. what about you?   
...                                                 ...   
3720    that's a good question. maybe it's not old age.   
3721                              are you right-handed?   
3722                                  yes. all my life.   
3723  you're wearing out your right hand. stop using...   
3724        but i do all my writing with my right hand.   

                                                Chatbot  
0                         i'm fine. how about yourself?  
1                   i'm pretty good. thanks for asking.  
2                     no problem. so how have you been?  
3                      i've been great. what about you?  
4

In [2]:
import string
import re
# importing regular expressions
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
# Lower case conversion
remove_n = lambda x: re.sub("\n", " ", x)
# removing \n and replacing them with empty value
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)
# removing non ascii characters
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
# removing alpha numeric values
df['User'] = df['User'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
# using map function and applying the function on query column
df['Chatbot'] = df['Chatbot'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
# using map function and applying the function on response column
print(df)
df.to_csv('modified_dataset.csv', index=False)


                                                   User  \
0                                hi  how are you doing    
1                         i m fine  how about yourself    
2                   i m pretty good  thanks for asking    
3                     no problem  so how have you been    
4                      i ve been great  what about you    
...                                                 ...   
3720    that s a good question  maybe it s not old age    
3721                              are you right handed    
3722                                  yes  all my life    
3723  you re wearing out your right hand  stop using...   
3724        but i do all my writing with my right hand    

                                                Chatbot  
0                         i m fine  how about yourself   
1                   i m pretty good  thanks for asking   
2                     no problem  so how have you been   
3                      i ve been great  what about you   
4

In [3]:
import torch
import spacy
import random
import torch.nn as nn
import torch.optim as optim

# Data Preprocessing
nlp = spacy.load("en_core_web_sm")
with open("modified_dataset.csv", "r") as file:
    lines = file.readlines()
    data = [line.split("\t") for line in lines]


In [ ]:
with open("modified_dataset.csv", "r") as file:
    words = file.read().split()
    print(words)
    unique_words = set(words)
    print(len(unique_words))
    print(unique_words)

In [ ]:
print(lines)

In [ ]:
print(data)

In [8]:
vocabulary = {"<PAD>": 0, "<UNK>": 1}  # Initialize with special tokens
with open("modified_dataset.csv", "r") as file:
    for line in file:
        words = line.strip().split()  # Split by whitespace for adapting the data format
        for word in words:
            if word not in vocabulary:
                vocabulary[word] = len(vocabulary)


In [ ]:
print(vocabulary)

In [10]:
def tokenize_and_convert_to_tensor(text, vocabulary, output_size):
    # Tokenize the text using spaCy
    tokens = [token.text for token in nlp(text)]

    # Convert tokens to indices using the provided vocabulary
    indices = [vocabulary.get(token, vocabulary["<UNK>"]) for token in tokens]
    indices = [min(idx, output_size - 1) for idx in indices]

    # Convert the list of indices to a PyTorch tensor
    tensor = torch.LongTensor(indices)

    return tensor


In [11]:
class SimpleChatbot(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleChatbot, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        embedded = self.embedding(input)
        output, hidden = self.gru(embedded)
        output = self.out(output)
        return output, hidden

In [12]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.4 MB/s eta 0:00:00


In [13]:
pip install flask